# Diaspora Event SDK - v3 API Example


In [ ]:
# Setup
# Install dependencies and initialize the client.

%pip install -e '.[kafka-python]'

import os
import uuid
import json
from datetime import datetime
from diaspora_event_sdk import Client as GlobusClient
from diaspora_event_sdk.sdk.kafka_client import KafkaProducer, KafkaConsumer

os.environ["DIASPORA_SDK_ENVIRONMENT"] = "local"
c = GlobusClient()


In [ ]:
# Key Management
# Demonstrate key creation, retrieval, and deletion. 
# Show how `get_key` retrieves from DynamoDB on subsequent calls.

# Create a new access key (replaces any existing key)
print("1. Creating a new access key:")
result = c.create_key_v3()
print(f"   Access Key: {result.get('access_key', 'N/A')[:20]}...")
print(f"   Message: {result.get('message', 'N/A')}")

# Get key multiple times - should retrieve from DynamoDB
print("\n2. Getting key again (should retrieve from DynamoDB):")
result = c.retrieve_key_v3()
print(f"   Access Key: {result.get('access_key', 'N/A')[:20]}...")
print(f"   Retrieved from DynamoDB: {result.get('retrieved_from_dynamodb', False)}")

# Delete key
print("\n3. Deleting access key:")
result = c.delete_key_v3()
print(f"   Message: {result.get('message', 'N/A')}")

# Get key again - should create a new one
print("\n4. Getting key after deletion (should create a new key):")
result = c.retrieve_key_v3()
print(f"   Access Key: {result.get('access_key', 'N/A')[:20]}...")
print(f"   Retrieved from DynamoDB: {result.get('retrieved_from_dynamodb', False)}")


In [ ]:
# Create Namespaces
# Create two unique namespaces using UUID
namespace1 = f"ns-{str(uuid.uuid4())[:8]}"
namespace2 = f"ns-{str(uuid.uuid4())[:8]}"

result1 = c.create_namespace_v3(namespace1)
print(f"Created namespace 1: {namespace1}")
print(f"  Result: {result1.get('message', 'N/A')}")

result2 = c.create_namespace_v3(namespace2)
print(f"\nCreated namespace 2: {namespace2}")
print(f"  Result: {result2.get('message', 'N/A')}")


In [ ]:
# Create Topics
# Create one topic under namespace1
topic1 = "topic-a"
result = c.create_topic_v3(namespace1, topic1)
print(f"Created topic '{topic1}' under '{namespace1}': {result.get('message', 'N/A')}")

# Create two topics under namespace2
topic2a = "topic-x"
topic2b = "topic-y"
result = c.create_topic_v3(namespace2, topic2a)
print(f"\nCreated topic '{topic2a}' under '{namespace2}': {result.get('message', 'N/A')}")
result = c.create_topic_v3(namespace2, topic2b)
print(f"Created topic '{topic2b}' under '{namespace2}': {result.get('message', 'N/A')}")


In [ ]:
# List Namespaces and Topics
# View all namespaces and their associated topics.

result = c.list_namespaces_v3()
print("All namespaces and topics:")
for ns, topics in result.get('namespaces', {}).items():
    print(f"  {ns}: {topics}")


In [ ]:
# Create a new access key (replaces any existing key)
print("1. Creating a new access key:")
result = c.create_key_v3()
print(f"   Access Key: {result.get('access_key', 'N/A')[:20]}...")
print(f"   Message: {result.get('message', 'N/A')}")

In [ ]:
# Produce and Consume Events
# Produce multiple messages to the first topic and then consume them.

# Kafka topic name format: "{namespace}.{topic}"
kafka_topic = f"{namespace1}.{topic1}"
print(kafka_topic)

# Produce multiple events using KafkaProducer (doesn't block on metadata)
p = KafkaProducer(kafka_topic)
for i in range(3):
    message = {
        "message_id": i + 1,
        "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "content": f"Message {i + 1} from v3 API"
    }
    future = p.send(kafka_topic, message)
    result = future.get(timeout=10)
    print(f"Produced message {i + 1}: offset={result.offset}")



In [ ]:
# Consume events
print("\nConsuming events:")
consumer = KafkaConsumer(kafka_topic, auto_offset_reset="earliest")
messages = consumer.poll(timeout_ms=10000)
for tp, msgs in messages.items():
    for message in msgs:
        data = json.loads(message.value.decode("utf-8"))
        print(f"  Consumed: {data}")

In [ ]:
# Cleanup
# Delete all topics and namespaces, then verify deletion.

# Delete topics from namespace1
result = c.delete_topic_v3(namespace1, topic1)
print(f"Deleted topic '{topic1}': {result.get('message', 'N/A')}")

# Delete topics from namespace2
result = c.delete_topic_v3(namespace2, topic2a)
print(f"Deleted topic '{topic2a}': {result.get('message', 'N/A')}")
result = c.delete_topic_v3(namespace2, topic2b)
print(f"Deleted topic '{topic2b}': {result.get('message', 'N/A')}")

# Delete namespaces
result = c.delete_namespace_v3(namespace1)
print(f"\nDeleted namespace '{namespace1}': {result.get('message', 'N/A')}")
result = c.delete_namespace_v3(namespace2)
print(f"Deleted namespace '{namespace2}': {result.get('message', 'N/A')}")


In [ ]:
# Verify Deletion
# List namespaces again to confirm all resources have been deleted.

result = c.list_namespaces_v3()
namespaces = result.get('namespaces', {})
if namespaces:
    print("Remaining namespaces and topics:")
    for ns, topics in namespaces.items():
        print(f"  {ns}: {topics}")
else:
    print("No namespaces found - cleanup successful!")
